In [0]:
import numpy as np
import pickle as pkl
import pandas as pd


from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
# from sklearn.model_selection import KFold
# from sklearn.model_selection import RandomizedSearchCV
# # from keras.wrappers.scikit_learn import KerasClassifier
# import keras
# from keras.models import Model
# from keras.layers import Lambda

# from keras.layers  import Dense, Input, Add, Flatten,multiply,Conv2D, MaxPool2D, Dropout, Reshape,Concatenate,Dot
# from keras.layers.embeddings import Embedding
# # from keras.callbacks import EarlyStopping

# from keras.callbacks import ModelCheckpoint
# # from keras.utils import plot_model
# from keras.backend import mean
# from keras.models import load_model
# # from keras.optimizers import Adamax
# # from keras.callbacks import LearningRateScheduler
# # from keras.callbacks import Callback
# # from keras.preprocessing import sequence

# # plt.style.use('ggplot')
# import tensorflow as tf

In [2]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


META Only

In [36]:
# with (open("/content/drive/My Drive/app1/data/data_Final_20052012.pkl", "rb")) as openfile:
#   data=pkl.load(openfile)
# data.i_perc[data.i_perc.isnull()]=1-data.r_perc-data.d_perc
# sum(data.bill_type.isnull())
# print(data.info())
# print(data.chamber.unique())#lower, upper
# print(data.bill_type.unique())#bill' 'resolution' 'joint_resolution' 'concurrent_resolution'
# print(data.party.unique())#d,r,i
# print(data.pip.unique())#d,r
# chamber=pd.get_dummies(data['chamber'],drop_first=True)
# bill_type=pd.get_dummies(data['bill_type'],drop_first=True)
# pip=pd.get_dummies(data['pip'],drop_first=True)
# party=pd.get_dummies(data['party'],drop_first=True)
## leg_index=pd.get_dummies(data['leg_index'],drop_first=True)

# data.drop(['chamber','bill_type','pip','party','leg_index','bill2idx','summary'],axis=1,inplace=True)

# data = pd.concat([data,chamber,bill_type,pip,party,leg_index],axis=1)
data.head()
#x:leg_index, bill_type, chamber, pip, party, total_sponsors, d_perc,i_perc,r_perc, y:labels
# n_l=len(data.leg_index.unique())
# labels=np.array(data["vote"].values.tolist())

# X_b=np.array(data["bill2idx"].values.tolist())
# X_l=np.array(data["leg_index"].values.tolist())

# pr=np.array(data["r_perc"].values.tolist())#685091
# pd=np.array(data["d_perc"].values.tolist())


# #Logistic regression

# # logit_model=sm.Logit(y,X)
# # result=logit_model.fit()
# # print(result.summary2())

# y_pred = logreg.predict(X_test)
# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

,natural_id,d_perc,i_perc,r_perc,total_sponsors,summary,vote,upper,concurrent_resolution,joint_resolution,resolution,r,i,r,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805
0,20072008r_HR 2900,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20072008r_HR 2900,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,1.0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20072008r_HR 2900,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20072008r_HR 2900,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20072008r_HR 2900,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,1.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<b>Out of Session testing

In [0]:
def test_model(model,test_data):
    with (open(test_file, "rb")) as openfile:
        test_data=pkl.load(openfile)
    X_b_test=np.array(test_data["bill2idx"].values.tolist())
    X_l_test=np.array(test_data["leg_index"].values.tolist())
    pr_test=np.array(test_data["r_perc"].values.tolist())
    pd_test=np.array(test_data["d_perc"].values.tolist())
    Y_test=np.array(test_data["vote"].values.tolist())
    del test_data
    loss,acc = model.evaluate([X_b_test,X_l_test,pr_test,pd_test], Y_test,verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], acc*100))
    return acc

# X_b_test2=np.array(data_test2["bill2idx"].values.tolist())
# X_l_test2=np.array(data_test2["leg_index"].values.tolist())
# Y_test2=np.array(data_test2["vote"].values.tolist())
# loss,acc = model.evaluate([X_b_test2,X_l_test2], Y_test2,verbose=False)
# print("%s: %.2f%%" % (model.metrics_names[1], acc*100))

In [0]:
# mini-batch generator
   
def data_generator(bill,leg,pr,pd, labels, batch_size, shuffle=True):
  data_size = len(labels)
#   num_batches_per_epoch = int((len(labels) - 1) / batch_size) + 1
  while True:
      # Shuffle the data at each epoch
      if shuffle:
          shuffle_indices = np.random.permutation(np.arange(data_size))
          shuffled_bill = bill[shuffle_indices]
          shuffled_leg = leg[shuffle_indices]
          shuffled_pr=pr[shuffle_indices]
          shuffled_pd=pd[shuffle_indices]
          shuffled_labels = labels[shuffle_indices]
      else:
          shuffled_bill = bill
          shuffled_leg = leg
          shuffled_labels = labels
          shuffled_pr = pr
          shuffled_pd = pd

      for batch_num in range(0, data_size, batch_size):
          
          start_index = batch_num# * batch_size
          end_index = batch_num+batch_size#min((batch_num + 1) * batch_size, data_size)
          X_b=shuffled_bill[start_index:end_index]
          X_l=shuffled_leg[start_index:end_index]
          pr=shuffled_pr[start_index:end_index]
          pd=shuffled_pd[start_index:end_index]
          X = [X_b, X_l, pr,pd]
          y = shuffled_labels[start_index: end_index]

          yield X, y


<b> Text Based Model 1: MWE

In [0]:
def model_MWE(n_leg,emb_dim=50,max_len=400):
    """
    Function creating the vote_prediction model's graph.
    
    Arguments:
    n_leg -- number of unique legislators
    max_len -- max words in a text
    
    Returns:
    model -- a model instance in Keras
    """
    bill_indices = Input(shape=(max_len,),dtype='int32',name="bill_in")
    
    with (open("/content/drive/My Drive/app1/data/emb_matrix.pkl","rb")) as openfile:
        emb_matrix=pkl.load(openfile)

    embedding_layer = Embedding(output_dim=emb_dim,input_dim=emb_matrix.shape[0],weights=[emb_matrix],input_length=max_len,trainable=False)
    del emb_matrix

    bill_emb = embedding_layer(bill_indices)
    
    #   Average the output of the Embedding layer over the word dimension
    bill_emb=Lambda(lambda x: mean(x, axis=1))(bill_emb)
    
    bill_emb=Dropout(0.2)(bill_emb)
    xb = Dense(25,kernel_initializer='glorot_uniform',bias_initializer='glorot_uniform',name="xb")(bill_emb)
    
    pr=Input(shape=(1,),dtype='float32',name='r_perc')
    pd=Input(shape=(1,),dtype='float32',name='d_perc')
    ar=Dense(25,use_bias=False, kernel_initializer='glorot_uniform')(pr)
    ar=Dropout(0.2)(ar)
    ad=Dense(25,use_bias=False, kernel_initializer='glorot_uniform')(pd)
    ad=Dropout(0.2)(ad)
    br=multiply([ar,xb])
    bd=multiply([ad,xb])
    xb=Add()([br,bd])
    
#     bill_emb = BatchNormalization()(bill_emb)
#     bill_emb=Dropout(0.2)(bill_emb)
    
    leg_in= Input(shape=(1,), dtype='int64', name='legislator_index')
    xl=Embedding(input_dim=n_leg,output_dim= 25, input_length=1,embeddings_initializer='glorot_uniform',mask_zero=False,name="xl",trainable=True)(leg_in)
#     xl = BatchNormalization()(xl)
    xl=Flatten()(xl)
    xl=Dropout(0.2)(xl)
    x=multiply([xb,xl])
    x=Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform',bias_initializer='glorot_uniform')(x)
    
    # Create Model instance which converts bill and leg id into vote prediction x.
    model = Model([bill_indices,leg_in,pr,pd],x)
    model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
    return model

<b> Testing in session

In [0]:
mc = ModelCheckpoint('/content/drive/My Drive/app1/best_model_MWE.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)
with (open("/content/drive/My Drive/app1/data/data_Final_20052012.pkl", "rb")) as openfile:
  data=pkl.load(openfile)
# sum(data.d_perc.isnull())
n_l=len(data.leg_index.unique())
labels=np.array(data["vote"].values.tolist())

X_b=np.array(data["bill2idx"].values.tolist())
X_l=np.array(data["leg_index"].values.tolist())

pr=np.array(data["r_perc"].values.tolist())#685091
pd=np.array(data["d_perc"].values.tolist())

x_b_train, x_b_valid, x_l_train, x_l_valid, pr_train, pr_valid, pd_train, pd_valid, y_train, y_valid = train_test_split(X_b,X_l,pr, pd,labels , test_size=0.05, stratify=labels)


model_in = model_MWE(n_l) 
# batch_size = 32

# train_gen = data_generator(x_b_train, x_l_train,pr_train,pd_train,y_train, batch_size)
# valid_gen = data_generator(x_b_valid,x_l_valid,pr_valid,pd_valid,y_valid, batch_size)
# num_batches_per_epoch = int((len(labels) - 1) / batch_size) + 1
del data,labels,X_b,X_l,pr,pd#,x_b_train,x_b_valid,x_l_train,x_l_valid,pr_train, pr_valid, pd_train,pd_valid,y_train,y_valid
# model_in.fit_generator(train_gen, samples_per_epoch=len(y_train),
#                               epochs=5, 
#                               validation_data=valid_gen,
#                               nb_val_samples=len(y_valid),
#                               callbacks=[mc])
history=model_in.fit([x_b_train,x_l_train,pr_train,pd_train], y_train,validation_data=([x_b_valid,x_l_valid,pr_valid,pd_valid], y_valid),initial_epoch = 0, epochs =25, batch_size = 32,verbose=0,callbacks=[mc])


Compile MWE Model

In [0]:
best_model = load_model(filepath='/content/drive/My Drive/app1/best_model_MWE.h5',custom_objects={'mean':mean})
test_file="/content/drive/My Drive/app1/data/data_Final_20132014.pkl"
test_model(best_model,test_file)

acc: 81.08%


0.8107655173283759

In [0]:
test_file="/content/drive/My Drive/app1/data/data_Final_20152016.pkl"
test_model(best_model,test_file)

acc: 72.31%


0.7230687010101857

In [0]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='test_loss')
plt.title('Training and validation Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['acc'], label='train_Accuracy')
plt.plot(history.history['val_acc'], label='test_Accuracy')
plt.title('Training and validation Accuracy')
plt.legend()
plt.show()

In [0]:
# kfold=KFold(3,True,42) #True: shuffle, 1: seed
# # # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=7,min_delta=0.1,baseline=.1)
# mc = ModelCheckpoint('best_model_MWE.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)
# with (open("/content/drive/My Drive/app1/data/data_Final_20052012.pkl", "rb")) as openfile:
#   data=pkl.load(openfile)
# data.head()
# n_l=len(data.leg_index.unique())
# split=kfold.split(data)

# train, test =next(split)    

# data_train=data.iloc[train]
# data_test=data.iloc[test]
# data_test=data_test[data_test.leg_index.isin(data_train.leg_index)]


# X_b_train=np.array(data_train["bill2idx"].values.tolist())
# X_l_train=np.array(data_train["leg_index"].values.tolist())
# Y_train=np.array(data_train["vote"].values.tolist())


# X_b_test=np.array(data_test["bill2idx"].values.tolist())
# X_l_test=np.array(data_test["leg_index"].values.tolist())
# Y_test=np.array(data_test["vote"].values.tolist())
# del data,data_train, data_test

# model_in = model_MWE(n_l)


,natural_id,bill_type,chamber,pip,d_perc,i_perc,r_perc,total_sponsors,summary,party,vote,leg_index,bill2idx
0,20072008r_HR 2900,bill,lower,d,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,d,1.0,0,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 2.0, 8.0, ..."
1,20072008r_HR 2900,bill,lower,d,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,r,1.0,1,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 2.0, 8.0, ..."
2,20072008r_HR 2900,bill,lower,d,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,d,1.0,2,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 2.0, 8.0, ..."
3,20072008r_HR 2900,bill,lower,d,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,d,1.0,3,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 2.0, 8.0, ..."
4,20072008r_HR 2900,bill,lower,d,0.727273,0.0,0.272727,33,Food and Drug Administration Amendments Act of...,d,1.0,4,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 2.0, 8.0, ..."


Train and Test MWT Model

In [0]:
model.save('MWTmodel_OutSession.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model identical to the previous one
# model2 = load_model('MWTmodel_OutSession.h5',custom_objects={'mean':mean})